In [3]:
from ultralytics import YOLO
import cv2
from sort import Sort
import numpy as np

In [4]:
model = YOLO("yolov8n.pt")
tracker = Sort()

In [6]:
cap = cv2.VideoCapture("train.mp4")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera not detected!")
        break

    # cv2.imshow("Webcam", frame)
    results = model(frame)
    person_only = frame.copy()
    
    height, width = frame.shape[:2]
    cv2.line(person_only, (0, height // 2), (width, height // 1), (0, 0, 255), 2)

    detections = []
    
    for box in results[0].boxes:
        class_id = int(box.cls[0])
        if class_id == 0:
            x1,y1,x2,y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            detections.append([x1,y1,x2,y2,conf])

    detections = np.array(detections)
    tracked_objects = tracker.update(detections)

    for obj in tracked_objects:
        x1,y1,x2,y2,obj_id = map(int, obj)
        cv2.rectangle(person_only, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(person_only, f"ID {obj_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)


    # annotated_frame = results[0].plot()
    # cv2.imshow("Webcame Detection", annotated_frame)

    cv2.imshow("Detecting Persons", person_only)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 12 persons, 1 bus, 1 truck, 106.1ms
Speed: 4.6ms preprocess, 106.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 1 bus, 1 truck, 109.7ms
Speed: 5.9ms preprocess, 109.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 1 bus, 94.9ms
Speed: 4.9ms preprocess, 94.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 1 bus, 1 truck, 71.3ms
Speed: 3.9ms preprocess, 71.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 1 bus, 1 truck, 69.6ms
Speed: 3.9ms preprocess, 69.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 1 bus, 1 truck, 110.2ms
Speed: 4.2ms preprocess, 110.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 1 bus, 1 truck, 72.9ms
Speed: 3.5ms preprocess, 72.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 